## Importiing libraries

In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow import keras

In [ ]:
img_dir = "./cell_images/cell_images"
#uninfected = "./cell_images/Uninfected"
#parasitized = "./cell_images/parasitized"
classes = os.listdir(img_dir)
print(classes)

## Generating the training and validation dataset from the image files

In [ ]:
size = 64
batch_size = 48
training_data = keras.preprocessing.image_dataset_from_directory(img_dir,
                                                                seed = 45,
                                                                validation_split = 0.2,
                                                                subset = "training",
                                                                image_size = (size,size),
                                                                batch_size = batch_size)

validation_data = keras.preprocessing.image_dataset_from_directory(img_dir,
                                                                seed = 45,
                                                                validation_split = 0.2,
                                                                subset = "validation",
                                                                image_size = (size,size),
                                                                batch_size = batch_size)


test_data = keras.preprocessing.image_dataset_from_directory(img_dir,
                                                                seed = 45,
                                                                validation_split = 0.2,
                                                                subset = "validation",
                                                                image_size = (size,size),
                                                                batch_size = batch_size)

## Plotting the images

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.figure(figsize = (12,8))
def view_images(target_dir, target_class):
    target_folder = target_dir + target_class
    
    random_image = random.sample(os.listdir(target_folder), 1)
    
    img = mpimg.imread(target_folder + '/' + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    
    print(f"Image shape: {img.shape}")
    return img
uninfected_images = view_images('./cell_images/cell_images/', 'Uninfected')

In [ ]:
parasitized_images = view_images('./cell_images/cell_images/', 'Parasitized')

## Creating the CNN architecture

In [ ]:
from tensorflow.keras.layers import Flatten, Conv2D, Dropout, BatchNormalization, MaxPool2D, Dense

In [ ]:
input = keras.layers.Input(shape = (64,64,3))
data_aug = keras.layers.experimental.preprocessing.Rescaling(1./255)(input)
conv1 = Conv2D(32, kernel_size = 3, activation = "relu")(data_aug)
pool1 = MaxPool2D(2)(conv1)
norm1 = BatchNormalization(axis = -1)(pool1)
drop1 = Dropout(0.2)(norm1)

conv2 = Conv2D(32, kernel_size = 3, activation = "relu")(drop1)
pool2 = MaxPool2D(2)(conv2)
norm2 = BatchNormalization(axis = -1)(pool2)
drop2 = Dropout(0.2)(norm2)

flatten = Flatten()(drop2)

hidden1 = Dense(512, activation = "relu")(flatten)
norm3 = BatchNormalization(axis = -1)(hidden1)
drop3 = Dropout(0.2)(norm3)

hidden2 = Dense(512, activation = "relu")(drop3)
norm4 = BatchNormalization(axis = -1)(hidden2)
drop4 = Dropout(0.2)(norm4)

output = Dense(2, activation = "sigmoid")(drop4)

model = keras.Model(inputs = input, outputs = output)

In [ ]:
model.compile(optimizer = "Adam",
             loss = "sparse_categorical_crossentropy",
             metrics = ["accuracy"])
print(model.summary())

In [ ]:
%%time
history = model.fit(training_data, epochs = 10, validation_data = validation_data, verbose = True)

In [ ]:
def plot_loss(history):

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # plot loss
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.title('Loss')
    plt.legend()

    plt.figure()
    plt.plot(epochs, accuracy, label='Training Accuracy')
    plt.plot(epochs, val_accuracy, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.title('Accuracy')
    plt.legend()

In [ ]:
plot_loss(history)

In [ ]:
pred = model.evaluate(validation_data)

In [ ]:
tf.keras.models.save_model(model,'my_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model = load_model('my_model.h5')

In [ ]:
img=image.load_img('./cell_images/Parasitized/1.png',target_size=(64,64))

In [ ]:
x = image.img_to_array(img)
x

In [ ]:
import numpy as np
x = x/255
x=np.expand_dims(x,axis=0)
#img_data=preprocess_input(x)
x.shape


In [ ]:
model.predict(x)

In [ ]:
a=np.argmax(model.predict(x), axis=1)

In [ ]:
a

In [ ]:
if(a==1):
    print("Uninfected")
else:
    print("Parasitized")